In [15]:
import sys
import pyscf
import os
from pyscf import dft
import numpy as np
sys.path.append('../../src/')
from apdft.calculator.gaussian import GaussianCalculator
# load local orbkit
sys.path.append("/mnt/c/Users/guido/workcopies/apdft/dep/orbkit/orbkit")
import orbkit

In [16]:
def _get_grid(nuclear_numbers, coordinates):
    """ Returns the default integration grid in Angstrom."""
    mol = pyscf.gto.Mole()
    for nuclear, coord in zip(nuclear_numbers, coordinates):
        # pyscf molecule init is in Angstrom
        mol.atom.extend([[nuclear, *coord]])
    mol.build()
    grid = dft.gen_grid.Grids(mol)
    grid.level = 3
    grid.build()
    # pyscf grid is in a.u.
    return grid.coords / (1 / 0.52917721067), grid.weights
def density_on_grid(inputfile, grid):
        orbkit.options.quiet = True
        orbkit.grid.x = grid[:, 0] * 1.88973
        orbkit.grid.y = grid[:, 1] * 1.88973
        orbkit.grid.z = grid[:, 2] * 1.88973
        orbkit.grid.is_initialized = True

        try:
            qc = orbkit.read.main_read(inputfile, itype="gaussian.fchk")
            rho = orbkit.core.rho_compute(qc, numproc=1)
        except:
            log.log(
                "Unable to read fchk file with orbkit.",
                level="error",
                filename=inputfile,
            )
            return grid[:, 0] * 0
        return rho

In [3]:
gridcoords, gridweights = _get_grid([7, 7], [[0., 0.,0.], [0.,0.,1.0]])

In [17]:
rho = density_on_grid('../../test/data/apdft-run/order-1/site-0-up/run.fchk', gridcoords)

In [18]:
(rho*gridweights).sum()

13.999914750656654

In [19]:
def get_modepn(coordinates, gridcoords, densweight):
    modepns = []
    for site in coordinates:
        ds = np.linalg.norm(gridcoords - site, axis=1)* ( 1 / 0.52917721067)
        
        modepns.append((densweight / ds).sum())
    return modepns

In [94]:
get_modepn(np.array([[0., 0.,0.], [0.,0.,1.0]]), gridcoords, density_on_grid('../../test/data/apdft-run/order-1/site-0-dn/run.fchk', gridcoords)*gridweights)

[21.601980474332226, 21.648818350378143]

In [96]:
GaussianCalculator.get_epn('../../test/data/apdft-run/order-1/site-0-dn/', np.array([[0., 0.,0.], [0.,0.,1.0]]), [0,1], [7, 7])

array([21.60206747, 21.67536947])

## prediction

In [33]:
get_dens = lambda _: density_on_grid('../../test/data/apdft-run/%s/run.fchk' % _, gridcoords)
rho_r = get_dens('order-0/site-all-cc')
d_rho_d_Z1 = get_dens('order-1/site-0-up') - get_dens('order-1/site-0-dn')
d_rho_d_Z2 = get_dens('order-1/site-1-up') - get_dens('order-1/site-1-dn')
d2_rho_dZ1 = get_dens('order-1/site-0-up') + get_dens('order-1/site-0-dn') - rho_r
d2_rho_dZ2 = get_dens('order-1/site-1-up') + get_dens('order-1/site-1-dn') - rho_r
d2_mixed = get_dens('order-2/site-0-1-up') + get_dens('order-2/site-0-1-dn') -d2_rho_dZ1-d2_rho_dZ2

In [39]:
signs = (-1, 1)
def eval_dV(rhotilde):
    E = 0.
    for site in (0, 1):
        ds = np.linalg.norm(gridcoords - np.array([0., 0., site]), axis=1)* ( 1 / 0.52917721067)
        E+= -signs[site]*(gridweights * rhotilde / ds).sum()
    return E
# order 0
print (0, eval_dV(rho_r))

# order 1
print (1, eval_dV(signs[0]*d_rho_d_Z1 / (2*2*0.05) + signs[1]*d_rho_d_Z2 / (2*2*0.05)))

# order 2
print (2, eval_dV((d2_rho_dZ1 / (3*0.05*2))+ (d2_rho_dZ2 / (3*0.05*2)) + (d2_mixed / (3*2*0.05*2))))

0 5.8262637168127185e-06
1 -0.9356666974103722
2 3.884918479002408e-05


In [104]:
def get_epn_coefficients(include_atoms, orders, deltaZ):
        """ EPN coefficients are the weighting of the electronic EPN from each of the finite difference calculations.
        
        The weights depend on the change in nuclear charge, i.e. implicitly on reference and target molecule as well as the finite difference stencil employed.
        TODO: Fix hard-coded weights from stencil."""
        # build alphas
        N = len(include_atoms)
        nvals = {0: 1, 1: N * 2, 2: N * (N - 1)}
        alphas = np.zeros(sum([nvals[_] for _ in orders]))

        # order 0
        if 0 in orders:
            alphas[0] = 1

        # order 1
        if 1 in orders:
            for siteidx in range(N):
                alphas[1 + siteidx * 2] += 5 * deltaZ[siteidx]
                alphas[1 + siteidx * 2 + 1] -= 5 * deltaZ[siteidx]
        #return alphas

        # order 2
        if 2 in orders:
            pos = 1 + N * 2 - 2
            for siteidx_i in range(N):
                for siteidx_j in range(siteidx_i, N):
                    if siteidx_i != siteidx_j:
                        pos += 2
                    if deltaZ[siteidx_j] == 0 or deltaZ[siteidx_i] == 0:
                        continue
                    if include_atoms[siteidx_j] > include_atoms[siteidx_i]:
                        prefactor = (
                            2 * (200 / 6.0) * deltaZ[siteidx_i] * deltaZ[siteidx_j]
                        )
                        alphas[pos] += prefactor
                        alphas[pos + 1] += prefactor
                        alphas[0] += 2 * prefactor
                        alphas[1 + siteidx_i * 2] -= prefactor
                        alphas[1 + siteidx_i * 2 + 1] -= prefactor
                        alphas[1 + siteidx_j * 2] -= prefactor
                        alphas[1 + siteidx_j * 2 + 1] -= prefactor
                    if include_atoms[siteidx_j] == include_atoms[siteidx_i]:
                        prefactor = (400 / 6.0) * deltaZ[siteidx_i] * deltaZ[siteidx_j]
                        alphas[0] -= 2 * prefactor
                        alphas[1 + siteidx_i * 2] += prefactor
                        alphas[1 + siteidx_j * 2 + 1] += prefactor

        return alphas
def _calculate_delta_Z_vector(numatoms, order, sites, direction):
        baseline = np.zeros(numatoms)

        if order > 0:
            sign = {"up": 1, "dn": -1}[direction] * 0.05
            baseline[list(sites)] += sign

        return baseline
def get_epn_matrix(include_atoms, coordinates, basepath, nuclear_numbers):
        """ Collects :math:`\int_Omega rho_i(\mathbf{r}) /|\mathbf{r}-\mathbf{R}_I|`. """
        N = len(include_atoms)

        coefficients = np.zeros((1 + N * 2 + N * (N - 1), N))

        # order 0
        pos = 0

        def get_epn(folder, order, direction, combination):
            res = 0.0
            charges = nuclear_numbers + _calculate_delta_Z_vector(
                        len(nuclear_numbers), order, combination, direction
                    )
            print (folder, len(nuclear_numbers), order, combination, direction)
            try:
                res = GaussianCalculator.get_epn(
                    folder,
                    coordinates,
                    include_atoms,
                    charges,
                )
            except ValueError:
                apdft.log.log(
                    "Calculation with incomplete results.",
                    level="error",
                    calulation=folder,
                )
            return res
        

        coefficients[pos, :] = get_epn("%s/order-0/site-all-cc/" % basepath, 0, 'up', 0)
        pos += 1

        # order 1
        for site in include_atoms:
            coefficients[pos, :] = get_epn(
                "%s/order-1/site-%d-up/" % (basepath, site), 1, 'up', [site]
            )
            coefficients[pos + 1, :] = get_epn(
                "%s/order-1/site-%d-dn/" % (basepath, site), 1, 'dn', [site]
            )
            pos += 2

        # order 2
        for site_i in include_atoms:
            for site_j in include_atoms:
                if site_j <= site_i:
                    continue

                coefficients[pos, :] = get_epn(
                    "%s/order-2/site-%d-%d-up/" % (basepath, site_i, site_j), 2, 'up', [site_i, site_j]
                )
                coefficients[pos + 1, :] = get_epn(
                    "%s/order-2/site-%d-%d-dn/" % (basepath, site_i, site_j), 2, 'dn', [site_i, site_j]
                )
                pos += 2

        return coefficients

In [105]:
alphas = get_epn_coefficients([0, 1], [0, 1, 2], signs)

In [106]:
mat = get_epn_matrix([0, 1], np.array([[0., 0.,0.], [0.,0.,1.0]]), '../../test/data/apdft-run/', [7, 7])

../../test/data/apdft-run//order-0/site-all-cc/ 2 0 0 up
../../test/data/apdft-run//order-1/site-0-up/ 2 1 [0] up
../../test/data/apdft-run//order-1/site-0-dn/ 2 1 [0] dn
../../test/data/apdft-run//order-1/site-1-up/ 2 1 [1] up
../../test/data/apdft-run//order-1/site-1-dn/ 2 1 [1] dn
../../test/data/apdft-run//order-2/site-0-1-up/ 2 2 [0, 1] up
../../test/data/apdft-run//order-2/site-0-1-dn/ 2 2 [0, 1] dn


In [107]:
# TODO: get_epn needs fractional nuclear charges for that calc
-np.sum(np.multiply(np.outer(alphas, signs), mat))

-0.9356627893298537

In [81]:
-1.2002513946645195/-0.9356666974103722

1.2827766532531655

In [82]:
mat

array([[21.62733747, 21.62733747],
       [21.65248147, 21.60575834],
       [21.60206747, 21.67536947],
       [21.60575834, 21.65248147],
       [21.67536947, 21.60206747],
       [21.63089634, 21.63089634],
       [21.62363561, 21.62363561]])

In [86]:
FNS='''%s/order-0/site-all-cc/
%s/order-1/site-0-up/
%s/order-1/site-0-dn/
%s/order-1/site-1-up/
%s/order-1/site-1-dn/
%s/order-2/site-0-1-up/
%s/order-2/site-0-1-dn/'''.split()

res = []
for fn in FNS:
    res.append((get_modepn(np.array([[0., 0.,0.], [0.,0.,1.0]]), gridcoords, density_on_grid((fn % '../../test/data/apdft-run')+'run.fchk', gridcoords)*gridweights)))

In [88]:
np.array(res) - mat

array([[-8.65190016e-05, -9.23452654e-05],
       [-8.67201438e-05, -9.23457747e-05],
       [-8.70003578e-05, -2.65511243e-02],
       [-8.64847071e-05, -9.24844793e-05],
       [-2.65453227e-02, -9.28812946e-05],
       [-8.71009919e-05, -9.29017143e-05],
       [-8.73277341e-05, -9.31810483e-05]])

In [89]:
np.array(res)

array([[21.62725096, 21.62724513],
       [21.65239475, 21.60566599],
       [21.60198047, 21.64881835],
       [21.60567185, 21.65238899],
       [21.64882415, 21.60197459],
       [21.63080923, 21.63080343],
       [21.62354829, 21.62354243]])

In [92]:
get_modepn(np.array([[0., 0.,0.], [0.,0.,1.0]]), gridcoords, density_on_grid(('../../test/data/apdft-run/order-1/site-0-dn/')+'run.fchk', gridcoords)*gridweights)

[21.601980474332226, 21.648818350378143]

In [97]:
mat

array([[21.62733747, 21.62733747],
       [21.65248147, 21.60575834],
       [21.60206747, 21.67536947],
       [21.60575834, 21.65248147],
       [21.67536947, 21.60206747],
       [21.63089634, 21.63089634],
       [21.62363561, 21.62363561]])